# Tutorial - strict_json

- Modified: 2 Apr 2025
- Note: We are going to be focusing on development of `parse_yaml` instead since it is more versatile and can output code more reliably. The structure of `parse_yaml` is similar to `strict_json`, with more configurable components
- `strict_json` will still be supported for legacy purposes
- JSON is the default output format for Functions, and is generated by `strict_json()`
- Works for JSON outputs with multiple ' or " or { or } or \ or unmatched braces/brackets that may break a json.loads()
- Reference Repo: https://github.com/tanchongmin/strictjson

## FAQ
- Q: Why not use a type-defined structured framework like Pydantic?
- A: Pydantic is very verbose in terms of defining the description and type for each field in the JSON, and can affect performance of LLM for longer context. Moreover, StrictJSON has very flexible checks which can be incorporated when generating the JSON.

# Setup Guide

## Step 1: Install StrictJSON

In [1]:
# !pip install strictjson

## Step 2: Import required functions and setup relevant API keys for your LLM

In [2]:
import os
from strictjson import *
from dotenv import load_dotenv
# put your secret keys in your .env
# For instance, if you are using OpenAI, your .env should contain
# export OPENAI_API_KEY = "sk-......."
load_dotenv()

True

## Step 3: Define your own LLM
- Define an `llm` function that takes in as inputs `system_prompt` and `user_prompt`, and outputs llm response string
- Note: If you do not define your own LLM, OpenAI `gpt-4o-mini` will be used as default

In [3]:
def llm(system_prompt: str, user_prompt: str) -> str:
    ''' Here, we use OpenAI for illustration, you can change it to your own LLM '''
    # ensure your LLM imports are all within this function
    from openai import OpenAI
    
    # define your own LLM here
    client = OpenAI()
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        temperature = 0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    return response.choices[0].message.content

In [4]:
# Verify that llm function is working
llm(system_prompt = 'You are a classifier to classify the sentiment of a sentence', 
    user_prompt = 'It is a hot and sunny day')

'Neutral'

# 1. Basic Generation

- **system_prompt**: Write in whatever you want the LLM to become. "You are a \<purpose in life\>"
- **user_prompt**: The user input. Later, when we use it as a function, this is the function input
- **output_format**: JSON of output variables in a dictionary, with the key as the output key, and the value as the output description
    - The output keys will be preserved exactly, while the LLM will generate content to match the description of the value as best as possible
- **llm**: The llm you want to use. Takes in `system_prompt` and `user_prompt` and outputs the LLM-generated string

#### Example LLM Definition
```python
def llm(system_prompt: str, user_prompt: str) -> str:
    ''' Here, we use OpenAI for illustration, you can change it to your own LLM '''
    # ensure your LLM imports are all within this function
    from openai import OpenAI
    
    # define your own LLM here
    client = OpenAI()
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        temperature = 0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    return response.choices[0].message.content
```

#### Example Usage
```python
res = strict_json(system_prompt = 'You are a classifier',
                    user_prompt = 'It is a beautiful and sunny day',
                    output_format = {'Sentiment': 'Type of Sentiment',
                                    'Adjectives': 'Array of adjectives',
                                    'Words': 'Number of words'},
                    llm = llm)
                                    
print(res)
```

#### Example Output
```{'Sentiment': 'Positive', 'Adjectives': ['beautiful', 'sunny'], 'Words': 7}```

In [5]:
res = strict_json(system_prompt = 'You are a classifier',
                    user_prompt = 'It is a beautiful and sunny day',
                    output_format = {'Sentiment': 'Type of Sentiment',
                                    'Adjectives': 'Array of adjectives',
                                    'Words': 'Number of words'},
                    llm = llm)
print(res)

{'Sentiment': 'Positive', 'Adjectives': ['beautiful', 'sunny'], 'Words': 6}


## Easy to split into corresponding elements

In [6]:
res['Sentiment']

'Positive'

In [7]:
res['Adjectives']

['beautiful', 'sunny']

In [8]:
res['Words']

6

# 2. Advanced Generation
- More advanced demonstration involving code that would typically break ```json.loads()```

#### Example Usage
```python
res = strict_json(system_prompt = 'You are a code generator, generating code to fulfil a task',
                    user_prompt = 'Given array p, output a function named func_sum to return its sum',
                    output_format = {'Elaboration': 'How you would do it',
                                     'C': 'Code',
                                    'Python': 'Code'},
                    llm = llm)
                                    
print(res)
```

#### Example Output
```{'Elaboration': 'Use a loop to iterate through each element in the array and add it to a running total.', ```

```'C': 'int func_sum(int p[], int size) {\n    int sum = 0;\n    for (int i = 0; i < size; i++) {\n        sum += p[i];\n    }\n    return sum;\n}', ```

```'Python': 'def func_sum(p):\n    sum = 0\n    for num in p:\n        sum += num\n    return sum'}```


In [9]:
res = strict_json(system_prompt = 'You are a code generator, generating code to fulfil a task',
                    user_prompt = 'Given array p, output a function named func_sum to return its sum',
                    output_format = {'Elaboration': 'How you would do it',
                                     'C': 'Code',
                                    'Python': 'Code'},
                    llm = llm)
                                    
print(res)

{'Elaboration': 'I will create a function named func_sum that takes an array p as an argument and returns the sum of its elements using the built-in sum function.', 'C': 'int func_sum(int p[], int size) { int sum = 0; for (int i = 0; i < size; i++) { sum += p[i]; } return sum; }', 'Python': 'def func_sum(p): return sum(p)'}


## Easy to split into corresponding elements

In [10]:
res['Elaboration']

'I will create a function named func_sum that takes an array p as an argument and returns the sum of its elements using the built-in sum function.'

In [11]:
print(res['C'])

int func_sum(int p[], int size) { int sum = 0; for (int i = 0; i < size; i++) { sum += p[i]; } return sum; }


In [12]:
print(res['Python'])

def func_sum(p): return sum(p)


In [13]:
## we can even run the Python code (potentially risky due to prompt injection attacks when running unverified code)
p = [1, 2, 3, 4, 5]
exec(res['Python'])
try:
    print('The output sum is', func_sum(p))
except Exception as e:
    print('An exception occured')

The output sum is 15


# 3. Type forcing output variables
- Generally, ```strict_json``` will infer the data type automatically for you for the output fields
- However, if you would like very specific data types, you can do data forcing using ```type: <data_type>``` at the last part of the output field description
- ```<data_type>``` must be of the form `int`, `float`, `str`, `dict`, `list`, `array`, `code`, `Dict[]`, `List[]`, `Array[]`, `Enum[]`, `bool` for type checking to work
- `code` removes all unicode escape characters that might interfere with normal code running
- The `Enum` and `List` are not case sensitive, so `enum` and `list` works just as well
- For `Enum[list_of_category_names]`, it is best to give an "Other" category in case the LLM fails to classify correctly with the other options.
- If `list` or `List[]` is not formatted correctly in LLM's output, we will correct it by asking the LLM to list out the elements line by line
- For `dict`,  we can further check whether keys are present using `Dict[list_of_key_names]`
- Other types will first be forced by rule-based conversion, any further errors will be fed into LLM's error feedback mechanism
- If `<data_type>` is not the specified data types, it can still be useful to shape the output for the LLM. However, no type checking will be done.
- Note: LLM understands the word `Array` better than `List` since `Array` is the official JSON object type, so in the backend, any type with the word `List` will be converted to `Array`.

### LLM-based checks
- If you would like the LLM to ensure that the type is being met, use `type: ensure <requirement>`
- This will run a LLM to check if the requirement is met. If requirement is not met, the LLM will generate what needs to be done to meet the requirement, which will be fed into the error-correcting loop of `strict_json`

#### Example Usage
```python
res = strict_json(system_prompt = 'You are a classifier',
                    user_prompt = 'It is a beautiful and sunny day',
                    output_format = {'Sentiment': 'Type of Sentiment, type: Enum["Pos", "Neg", "Other"]',
                                    'Adjectives': 'Array of adjectives, type: List[str]',
                                    'Words': 'Number of words, type: int',
                                    'In English': 'Whether sentence is in English, type: bool'},
                  llm = llm)
                                    
print(res)
```

#### Example Output
```{'Sentiment': 'Pos', 'Adjectives': ['beautiful', 'sunny'], 'Words': 7, 'In English': True}```

#### Example Usage
```python
res = strict_json(system_prompt = 'You are an expert at organising birthday parties',
                    user_prompt = 'Give me some information on how to organise a birthday',
                    output_format = {'Famous Quote about Age': 'type: ensure quote contains the word age',
                                    'Lucky draw numbers': '3 numbers from 1-50, type: List[int]',
                                    'Sample venues': 'Describe two venues, type: List[Dict["Venue", "Description"]]'},
                    llm = llm)

print(res)
```

#### Example Output
`Using LLM to check "The secret of staying young is to live honestly, eat slowly, and lie about your age. - Lucille Ball" to see if it adheres to "quote contains the word age" Requirement Met: True`


```{'Famous Quote about Age': 'The secret of staying young is to live honestly, eat slowly, and lie about your age. - Lucille Ball',```
```'Lucky draw numbers': [7, 21, 35],```

```'Sample venues': [{'Venue': 'Beachside Resort', 'Description': 'A beautiful resort with stunning views of the beach. Perfect for a summer birthday party.'}, {'Venue': 'Indoor Trampoline Park', 'Description': 'An exciting venue with trampolines and fun activities. Ideal for an active and energetic birthday celebration.'}]}```

In [14]:
res = strict_json(system_prompt = 'You are a classifier',
                    user_prompt = 'It is a beautiful and sunny day',
                    output_format = {'Sentiment': 'Type of Sentiment, type: Enum["Pos", "Neg", "Other"]',
                                    'Adjectives': 'Array of Adjectives, type: List[str]',
                                    'Words': 'Number of words, type: int',
                                    'In English': 'Whether sentence is in English, type: bool'},
                    llm = llm)

print(res)

{'Sentiment': 'Pos', 'Adjectives': ['beautiful', 'sunny'], 'Words': 7, 'In English': True}


In [15]:
# Example of multiple nested inputs with multiple types of checks
# Quotes may fail json parsing, so let us use LLMs to fix it
res = strict_json(system_prompt = 'You are an expert at organising birthday parties',
                    user_prompt = 'Give me some information on how to organise a birthday',
                    output_format = {'Famous Quote about Age': 'type: ensure quote includes word age',
                                    'Lucky draw numbers': '3 numbers from 1-50, type: List[int]',
                                    'Sample venues': 'Describe two venues, type: List[Dict["Venue", "Description"]]'},
                    llm = llm)

print(res)

Using LLM to check "Age is merely the number of years the world has been enjoying you." to see if it adheres to "quote includes word age"
{'Famous Quote about Age': 'Age is merely the number of years the world has been enjoying you.', 'Lucky draw numbers': [7, 23, 35], 'Sample venues': [{'Venue': 'Local Park', 'Description': 'A spacious outdoor area perfect for picnics and games, ideal for a casual birthday celebration.'}, {'Venue': 'Community Center', 'Description': 'A versatile indoor space that can be decorated according to the theme of the party, suitable for larger gatherings.'}]}


# 4. Nested Outputs
- StrictJSON supports nested outputs like nested lists and dictionaries

#### Example Input
```python
res = strict_json(system_prompt = 'You are a classifier',
                    user_prompt = 'It is a beautiful and sunny day',
                    output_format = {'Sentiment': ['Type of Sentiment', 
                                                   'Strength of Sentiment, type: Enum[1, 2, 3, 4, 5]'],
                                    'Adjectives': "Name and Description as separate keys, type: List[Dict['Name', 'Description']]",
                                    'Words': {
                                        'Number of words': 'Word count', 
                                        'Language': {
                                              'English': 'Whether it is English, type: bool',
                                              'Chinese': 'Whether it is Chinese, type: bool'
                                                  },
                                        'Proper Words': 'Whether the words are proper in the native language, type: bool'
                                        }
                                    },
                 llm = llm)

print(res)
```

#### Example Output
`{'Sentiment': ['Positive', 3],`

`'Adjectives': [{'Name': 'beautiful', 'Description': 'pleasing to the senses'}, {'Name': 'sunny', 'Description': 'filled with sunshine'}],`

`'Words':`

`     {'Number of words': 6,`
    
`     'Language': {'English': True, 'Chinese': False},`

`     'Proper Words': True}`
    
`}`

In [16]:
res = strict_json(system_prompt = 'You are a classifier',
                    user_prompt = 'It is a beautiful and sunny day',
                    output_format = {'Sentiment': ['Type of Sentiment', 
                                                   'Strength of Sentiment, type: Enum[1, 2, 3, 4, 5]'],
                                    'Adjectives': "Name and Description as separate keys, type: List[Dict['Name', 'Description']]",
                                    'Words': {
                                        'Number of words': 'Word count', 
                                        'Language': {
                                              'English': 'Whether it is English, type: bool',
                                              'Chinese': 'Whether it is Chinese, type: bool'
                                                  },
                                        'Proper Words': 'Whether the words are proper in the native language, type: bool'
                                        }
                                    },
                 llm = llm)
print(res)

{'Sentiment': ['Positive', 5], 'Adjectives': [{'Name': 'Beautiful', 'Description': 'Pleasing the senses or mind aesthetically'}, {'Name': 'Sunny', 'Description': 'Bright with sunlight'}], 'Words': {'Number of words': 7, 'Language': {'English': True, 'Chinese': False}, 'Proper Words': True}}


# 5. Return as JSON
- By default, `strict_json` returns a Python Dictionary
- If needed to parse as JSON, simply set `return_as_json=True`
- By default, this is set to `False` in order to return a Python Dictionry

In [17]:
res = strict_json(system_prompt = 'You are a classifier',
                    user_prompt = 'It is a beautiful and sunny day',
                    output_format = {'Sentiment': ['Type of Sentiment', 
                                                   'Strength of Sentiment, type: Enum[1, 2, 3, 4, 5]'],
                                    'Adjectives': "Name and Description as separate keys, type: List[Dict['Name', 'Description']]",
                                    'Words': {
                                        'Number of words': 'Word count', 
                                        'Language': {
                                              'English': 'Whether it is English, type: bool',
                                              'Chinese': 'Whether it is Chinese, type: bool'
                                                  },
                                        'Proper Words': 'Whether the words are proper in the native language, type: bool'
                                        }
                                    },
                 llm = llm,
                 return_as_json = True) # Toggle this to True (Note: Also works with OpenAI JSON Mode)
print(res)

{"Sentiment": ["Positive", 5], "Adjectives": [{"Name": "Beautiful", "Description": "Pleasing the senses or mind aesthetically"}, {"Name": "Sunny", "Description": "Bright with sunlight"}], "Words": {"Number of words": 7, "Language": {"English": true, "Chinese": false}, "Proper Words": true}}


# 6. Integrating with OpenAI JSON Mode
- Do note that you will require the `openai` Python package for this
- If you want to use the OpenAI JSON Mode, you can simply add in ```openai_json_mode = True``` and set ```model = 'gpt-4-1106-preview'``` or ```model = 'gpt-3.5-turbo-1106'``` in ```strict_json``` or ```Function```
- We will set model to ```gpt-3.5-turbo-1106``` by default if you provide an invalid model
- This does not work with the `llm` variable
- Note that type checking does not work with OpenAI JSON Mode

#### Example Usage
```python
res = strict_json(system_prompt = 'You are a classifier',
                    user_prompt = 'It is a beautiful and sunny day',
                    output_format = {'Sentiment': 'Type of Sentiment',
                                    'Adjectives': 'Array of adjectives',
                                    'Words': 'Number of words'},
                    model = 'gpt-3.5-turbo-1106' # Set the model
                    openai_json_mode = True) # Toggle this to True
                                    
print(res)
```

#### Example Output
```{'Sentiment': 'positive', 'Adjectives': ['beautiful', 'sunny'], 'Words': 6}```

In [18]:
res = strict_json(system_prompt = 'You are a classifier',
                user_prompt = 'It is a beautiful and sunny day',
                output_format = {'Sentiment': 'Type of Sentiment',
                                'Adjectives': 'Array of adjectives',
                                'Words': 'Number of words'},
               model = 'gpt-3.5-turbo-1106', # Set the model
               openai_json_mode = True) # Toggle this to True
print(res)

{'Sentiment': 'Positive', 'Adjectives': ['beautiful', 'sunny'], 'Words': 6}


# 7. Async Mode

- `AsyncFunction` and `strict_json_async`
    - These are the async equivalents of `Function` and `strict_json`
    - You will need to define an LLM that can operate in async mode
    - Everything is the same as the sync version of the functions, except you use the `await` keyword when calling `AsyncFunction` and `strict_json_async`
    
    
- Using Async can help do parallel processes simulataneously, resulting in a much faster workflow

#### Example LLM in Async Mode
```python
async def llm_async(system_prompt: str, user_prompt: str):
    ''' Here, we use OpenAI for illustration, you can change it to your own LLM '''
    # ensure your LLM imports are all within this function
    from openai import AsyncOpenAI
    
    # define your own LLM here
    client = AsyncOpenAI()
    response = await client.chat.completions.create(
        model='gpt-4o-mini',
        temperature = 0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    return response.choices[0].message.content
```

#### Example Input (strict_json_async)
```python
res = await strict_json_async(system_prompt = 'You are a classifier',
                    user_prompt = 'It is a beautiful and sunny day',
                    output_format = {'Sentiment': 'Type of Sentiment',
                                    'Adjectives': 'Array of adjectives',
                                    'Words': 'Number of words'},
                                     llm = llm_async) # use an async LLM function

print(res)
```

#### Example Output
`{'Sentiment': 'Positive', 'Adjectives': ['beautiful', 'sunny'], 'Words': 7}`

#### Example Input (AsyncFunction)
```python
fn =  AsyncFunction(fn_description = 'Output a sentence with <obj> and <entity> in the style of <emotion>', 
                     output_format = {'output': 'sentence'})

res = await fn('ball', 'dog', 'happy') #obj, entity, emotion

print(res)
```

#### Example Output
`{'output': 'The dog happily chased the ball.'}`


In [19]:
async def llm_async(system_prompt: str, user_prompt: str):
    ''' Here, we use OpenAI for illustration, you can change it to your own LLM '''
    # ensure your LLM imports are all within this function
    from openai import AsyncOpenAI
    
    # define your own LLM here
    client = AsyncOpenAI()
    response = await client.chat.completions.create(
        model='gpt-4o-mini',
        temperature = 0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    return response.choices[0].message.content

In [20]:
# Demonstration of strict_json_async
res = await strict_json_async(system_prompt = 'You are a classifier',
                    user_prompt = 'It is a beautiful and sunny day',
                    output_format = {'Sentiment': 'Type of Sentiment',
                                    'Adjectives': 'Array of adjectives',
                                    'Words': 'Number of words'},
                                     llm = llm_async) # use an async LLM function

print(res)

{'Sentiment': 'Positive', 'Adjectives': ['beautiful', 'sunny'], 'Words': 6}


# 8. Additional Output Field Checks (Advanced)

- You can also specify your own custom check function that will be used to check the output field (which will be in `str`, `int`, `float`, `list` or `dict` format inferred by LLM or specified in `type: <data type>`)
- Ensure that what you are checking for is implied in the output field's description in `output_format` of `strict_json` or `Function`
- Your custom check function must take in: `output_field`
- Your custom check function must output: 
    - `requirement` (str): The requirement you are checking for
    - `requirement_met` (bool): Whether condition is met, True or False
    - `action_needed` (str): What needs to be done to meet requirement if requirement_met is False
- If `requirement_met` is False, the `requirement` and `action_needed` message will be used for the `strict_json` error correcting mechanism. Otherwise, the error correcting mechanism will not be triggered
- `action_needed` is used to tell the LLM what it needs to do to meet your requirements (LLM is not able to self-correct without guidance for most cases). Try to be as specific as possible to improve error correction success rate.
- Pass in your custom check function inside `custom_checks` variable of `strict_json` or `Function` under the same key as that in `output_format`
- You can add multiple check functions for one variable by putting it inside the same list
- Example custom check function named `hello_world_check` which checks for the presence of hello world
- You can also use information in the variable `check_data` for checks (input via `strict_json` or `Function`)

#### Example Custom Check Functions
```python
def hello_world_check(output_field, check_data) -> (str, bool, str):
    ''' Example function 1: Checks whether hello world is present in output_field. '''
    requirement = 'Check whether hello world is present in output field'
    requirement_met = True
    action_needed = ''
    # do a check for requirement of having 'hello'
    if 'hello' not in str(output_field):
        requirement_met = False
        action_needed += 'Add in the word hello into output field, '
    if 'world' not in str(output_field):
        requirement_met = False
        action_needed += 'Add in the word world into output field, '
    return (requirement, requirement_met, action_needed)
```

```python
def function_name_check(output_field, check_data) -> (str, bool, str):
    ''' Example function 2: Checks whether function name is present in output_field
    Uses additional information from the check_data variable of strict_json'''
    function_name = check_data['Function name']
    requirement = f'Check whether {function_name} is present in output field'
    requirement_met = True
    action_needed = ''
    
    # do a check for requirement of having 'myprint'
    if function_name not in str(output_field):
        requirement_met = False
        action_needed += f'Ensure that function name "{function_name}" is used, '
    return (requirement, requirement_met, action_needed)
```

#### Example Usage
```python
# we can input our custom_checks as a list of check functions, and check_data is the additional information for these check functions
res = strict_json(system_prompt = 'You are a code generator',
                    user_prompt = 'Print out hello world',
                    output_format = {'Thoughts': 'How to do it',
                                    'Python Code': 'Function beginning with def myprint() -> str:, type: code'},
                    llm = llm,
                    custom_checks = {'Python Code': [hello_world_check, function_name_check]}, # add in custom_checks
                    check_data = {'Function name:' 'myprint'}) # add in check_data
                                    
print(res)
```
#### Example Output
`Running check for "Check whether hello world is present in output field" on output field of "Python Code"
Requirement met`


`Running check for "Check whether myprint is present in output field" on output field of "Python Code"
Requirement met`


`{'Thoughts': 'To print out "hello world", use the print() function in Python.',`
`'Python Code': 'def myprint() -> str:\n    return "hello world"'}`

In [21]:
def hello_world_check(output_field, check_data) -> (str, bool, str):
    ''' Example function 1: Checks whether hello world is present in output_field. '''
    requirement = 'Check whether hello world is present in output field'
    requirement_met = True
    action_needed = ''
    # do a check for requirement of having 'hello'
    if 'hello' not in str(output_field):
        requirement_met = False
        action_needed += 'Add in the lowercase word hello into output field, '
    if 'world' not in str(output_field):
        requirement_met = False
        action_needed += 'Add in the lowercase word world into output field, '
    return (requirement, requirement_met, action_needed)

def function_name_check(output_field, check_data) -> (str, bool, str):
    ''' Example function 2: Checks whether function name is present in output_field
    Uses additional information from the check_data variable of strict_json'''
    function_name = check_data['Function name']
    requirement = f'Check whether {function_name} is present in output field'
    requirement_met = True
    action_needed = ''
    
    # do a check for requirement of having 'myprint'
    if function_name not in str(output_field):
        requirement_met = False
        action_needed += f'Ensure that function name "{function_name}" is used, '
    return (requirement, requirement_met, action_needed)

In [22]:
# we can input our custom_checks as a list of check functions, and check_data is the additional information for these check functions
res = strict_json(system_prompt = 'You are a code generator',
                    user_prompt = 'Print out hello world',
                    output_format = {'Thoughts': 'How to do it',
                                    'Python Code': 'Function beginning with def myprint() -> str:, type: code'},
                    llm = llm,
                    custom_checks = {'Python Code': [hello_world_check, function_name_check]},
                    check_data = {'Function name': 'myprint'})
                                    
print(res)

Running check for "Check whether hello world is present in output field" on output field of "Python Code"
Requirement met


Running check for "Check whether myprint is present in output field" on output field of "Python Code"
Requirement met


{'Thoughts': 'To print out "hello world", I will define a function that uses the print statement in Python.', 'Python Code': 'def myprint() -> str:\n    print("hello world")'}


# Optional: Under the hood (Explanation of how strict_json works)
- When given the output JSON format, it adds a delimiter (default: ###) to enclose the key of the JSON.
- Example Output JSON provided: ```{'Sentiment': 'Type of Sentiment'}```
- Example Output JSON interpreted by Strict JSON: ```{'###Sentiment###': 'Type of Sentiment'}```
- We then process the JSON format by using regex to search for the delimiter to extract the keys and values
- Works for nested data structures as well by extracting recursively
- Note: Change the delimiter to whatever is not present in your dataset

In [23]:
# a very difficult chunk of text for json.loads() to parse (it will fail)
res = '''{
'###Question of the day###': `What is the 'x' in dx/dy?`, 
'###Code Block 1###':   '#include <stdio.h>\nint main(){\nint x = 'a'; return 0;\n}',
`###Optional Block###`: "",
"###Another Code###": 'import numpy as np
### Oh what is this doing here
print("It can handle so many quotations ' \\" and backslashes and unexpected curly braces { } You don't even need to match }!")',
'###Some characters###': '~!@#$%^&*()_+-'"{}[];?><,.'
}'''

In [24]:
# change this to whatever is not common in your dataset
delimiter = '###'

In [25]:
import re
# Use regular expressions to extract keys and values
pattern = fr",*\s*[\'\"`]{delimiter}([^#]*){delimiter}[\'\"`]:\s*"

matches = re.split(pattern, str(res[1:-1]).strip())

# remove null matches
my_matches = [match for match in matches if match !='']

print(my_matches)

['Question of the day', "`What is the 'x' in dx/dy?`", 'Code Block 1', "'#include <stdio.h>\nint main(){\nint x = 'a'; return 0;\n}'", 'Optional Block', '""', 'Another Code', '\'import numpy as np\n### Oh what is this doing here\nprint("It can handle so many quotations \' \\" and backslashes and unexpected curly braces { } You don\'t even need to match }!")\'', 'Some characters', '\'~!@#$%^&*()_+-\'"{}[];?><,.\'']


In [26]:
# remove the ' from the value matches
curated_matches = [match[1:-1] if match[0] in '\'"`' else match for match in my_matches]

print(curated_matches)

['Question of the day', "What is the 'x' in dx/dy?", 'Code Block 1', "#include <stdio.h>\nint main(){\nint x = 'a'; return 0;\n}", 'Optional Block', '', 'Another Code', 'import numpy as np\n### Oh what is this doing here\nprint("It can handle so many quotations \' \\" and backslashes and unexpected curly braces { } You don\'t even need to match }!")', 'Some characters', '~!@#$%^&*()_+-\'"{}[];?><,.']


In [27]:
len(curated_matches)

10

In [28]:
# create a dictionary
end_dict = {}
for i in range(0, len(curated_matches), 2):
    end_dict[curated_matches[i]] = curated_matches[i+1]
    
print(end_dict)

{'Question of the day': "What is the 'x' in dx/dy?", 'Code Block 1': "#include <stdio.h>\nint main(){\nint x = 'a'; return 0;\n}", 'Optional Block': '', 'Another Code': 'import numpy as np\n### Oh what is this doing here\nprint("It can handle so many quotations \' \\" and backslashes and unexpected curly braces { } You don\'t even need to match }!")', 'Some characters': '~!@#$%^&*()_+-\'"{}[];?><,.'}


In [29]:
for key, value in end_dict.items():
    print('Key:', key)
    print('Value:', value)
    print('#####')

Key: Question of the day
Value: What is the 'x' in dx/dy?
#####
Key: Code Block 1
Value: #include <stdio.h>
int main(){
int x = 'a'; return 0;
}
#####
Key: Optional Block
Value: 
#####
Key: Another Code
Value: import numpy as np
### Oh what is this doing here
print("It can handle so many quotations ' \" and backslashes and unexpected curly braces { } You don't even need to match }!")
#####
Key: Some characters
Value: ~!@#$%^&*()_+-'"{}[];?><,.
#####
